In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# 加载数据
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
# 数据预处理
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    x = tf.reshape(x, [28 * 28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x, y
    

In [4]:
# 划分训练接和测试集
batch_size = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).batch(batch_size)

In [5]:
# 验证数据类型
sample = next(iter(db_train))
print(sample[0].shape, sample[1].shape)

(128, 784) (128, 10)


In [7]:
class MyDense(keras.layers.Layer):
    """
    自定义层 __init__,call方法必须实现
    """
    def __init__(self, input_dim, out_dim):
        super(MyDense, self).__init__()
        self.kernel = self.add_variable('w', [input_dim, out_dim])
        self.bias = self.add_variable('b', [out_dim])
    
    def call(self, input_data, training=None):
        out = input_data@self.kernel + self.bias
        return out
    
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = MyDense(28 * 28, 256)
        self.dense2 = MyDense(256, 128)
        self.dense3 = MyDense(128, 64)
        self.dense4 = MyDense(64, 32)
        self.dense5 = MyDense(32, 10)
    
    def call(self, input_data, training=None):
        x = self.dense1(input_data)
        x = tf.nn.relu(x)
        
        x = self.dense2(x)
        x = tf.nn.relu(x)
                
        x = self.dense3(x)
        x = tf.nn.relu(x)
                
        x = self.dense4(x)
        x = tf.nn.relu(x)
                
        x = self.dense5(x)
                
        return x
        
        
        
        

In [8]:
# 建立自定义的模型
model = MyModel()

model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
model.fit(db_train, epochs=10, validation_data=db_test)

Epoch 1/10
469/469 [==============================] - 7s 15ms/step - loss: 0.5559 - accuracy: 0.7086 - val_loss: 0.4315 - val_accuracy: 0.8464
Epoch 2/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3726 - accuracy: 0.8617 - val_loss: 0.3908 - val_accuracy: 0.8588
Epoch 3/10
469/469 [==============================] - 6s 12ms/step - loss: 0.3342 - accuracy: 0.8747 - val_loss: 0.3748 - val_accuracy: 0.8635
Epoch 4/10
469/469 [==============================] - 5s 12ms/step - loss: 0.3086 - accuracy: 0.8834 - val_loss: 0.3686 - val_accuracy: 0.8647
Epoch 5/10
469/469 [==============================] - 6s 12ms/step - loss: 0.2896 - accuracy: 0.8917 - val_loss: 0.3795 - val_accuracy: 0.8642
Epoch 6/10
469/469 [==============================] - 6s 13ms/step - loss: 0.2731 - accuracy: 0.8965 - val_loss: 0.3817 - val_accuracy: 0.8655
Epoch 7/10
469/469 [==============================] - 6s 12ms/step - loss: 0.2615 - accuracy: 0.9010 - val_loss: 0.3652 - val_accuracy: 0.8691

In [9]:
model.evaluate(db_test)

79/79 [==============================] - 1s 10ms/step - loss: 0.3499 - accuracy: 0.8758


[0.3499346147232418, 0.8758]